## Identify the missing ones : 

> All the depos in 77 phages : 

In [8]:
import os 
import pandas as pd
from collections import Counter

path_classic="/media/concha-eloko/Linux/77_strains_phage_project"

to_discard = ["K80PH1317a"]

rbp_df = pd.read_csv(f"{path_classic}/RBP_77_phages.tsv", sep = "\t", header =0).fillna("None")
rbp_df = rbp_df[~rbp_df["Phage"].isin(to_discard)]
depo_df = rbp_df[rbp_df["Function"] == "depolymerase"]


In [9]:
depos_format_all = []
for _, row in depo_df.iterrows() :
    cds_name = "_".join(row["CDS"].split(" "))
    depo_name = f'{row["Phage"]}__{cds_name}'
    depos_format_all.append(depo_name)


In [10]:
path_ppt = "/media/concha-eloko/Linux/PPT_clean"

current_depo = pd.read_csv(f"{path_ppt}/77_current_dpos.tsv", names = ["depos"])
depo_current_format = current_depo["depos"].tolist()

In [11]:
missing_ones = set(depos_format_all) - set(depo_current_format)
len(missing_ones)

21

***
## Create a multi fasta file : 

In [13]:
path_project = "/media/concha-eloko/Linux/77_strains_phage_project"
path_annotation = f"{path_project}/annotation_classic"

missing_depo_df = pd.read_csv(f"{path_ppt}/missing_dpos_delination.tsv", names = ["depos", "boundaries"], sep = "\t")


In [14]:
missing_depo_df

,depos,boundaries
0,K10PH82C1__cds_49,250-1321
1,K13PH07C1L__cds_10,41-537
2,K13PH07C1L__cds_11,124-309
3,K13PH07C1L__cds_12,1-203
4,K15PH90__cds_54,177-684
5,K18PH07C1__cds_243,123-416
6,K18PH07C1__cds_245,full
7,K22PH164C1__cds_11,100-564
8,K30lambda2_2__cds_152,full
9,K30lambda2_2__cds_20,253-400


In [31]:
with open(f"{path_ppt}/missing_depo.24062024.fasta", "w") as outfile :
    for _, row in missing_depo_df.iterrows() :
        if row["boundaries"] != "ignore" :
            phage = row["depos"].split("__")[0]
            cds = row["depos"].split("_")[-1]
            faa_file = open(f"{path_classic}/annotation_29092023/{phage}/{phage}.multi.annotated.faa").read().split(">")
            if row["boundaries"] == "full" :
                seq = faa_file[int(cds)].split('\n')[1]
            else :
                start , end = int(row["boundaries"].split("-")[0]) , int(row["boundaries"].split("-")[1])
                seq = faa_file[int(cds)].split('\n')[1][start : end]
            outfile.write(f">{phage}__cds_{cds}\n{seq}\n\n")


> Fix missing some missing dpo

In [6]:
from Bio import SeqIO
path_ppt = "/media/concha-eloko/Linux/PPT_clean"

missing_depo_seqio = {record.description : record.seq for record in SeqIO.parse(f"{path_ppt}/missing_depo.24062024.fasta", "fasta")}
len(missing_depo_seqio)

19

> Move it to the server :

In [ ]:
rsync -avzhe ssh \
/media/concha-eloko/Linux/PPT_clean/missing_depo.24062024.fasta \
conchae@garnatxa.srv.cpd:/home/conchae/PhageDepo_pdb

rsync -avzhe ssh \
conchae@garnatxa.srv.cpd:/home/conchae/77_strains_phage_project/missing_depo.24062024.fasta.esm2_out \
/media/concha-eloko/Linux/77_strains_phage_project/


In [2]:
import torch
import os 
import pandas as pd
from tqdm import tqdm 


path_esm = "/media/concha-eloko/Linux/77_strains_phage_project/missing_depo.24062024.fasta.esm2_out"

embeddings_esm = {}
for file in tqdm(os.listdir(path_esm)) :
    path_file = f"{path_esm}/{file}"
    index = file.split(".")[0]
    embb = torch.load(f"{path_file}")["mean_representations"][33].tolist()
    embeddings_esm[index] = embb
    
with open(f"/media/concha-eloko/Linux/77_strains_phage_project/missed_dpo.77_phages.domains.24062024.esm2.embedding.csv" , "w") as outfile :
    for index in tqdm(embeddings_esm) :
        outfile.write(f"{index},")
        #for _,  emb in enumerate(embeddings_esm[index]) :
        #    outfile.write(f"{emb},")
        data_emb = ",".join([str(item) for item in embeddings_esm[index]])
        outfile.write(f"{data_emb}\n")
        #outfile.write("\n")

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 19/19 [00:00<00:00, 1268.57it/s]
